# 本节学习
1. 如何在GPU上训练
2. 如何在多GPU上并行训练

## 1. 如何在GPU上训练

In [2]:
import torch
# 找GPU设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) # Assume that we are on a CUDA machine, then this should print a CUDA device, cuda:0

cpu


In [7]:
# 将网络、输入、输出转换为CUDA张量
net.to(device)
inputs, labels = inputs.to(device), labels.to(device)

NameError: name 'inputs' is not defined

## 2. 如何在多GPU上并行训练——数据并行
PyTorch 默认只会使用一个 GPU。通过使用 DataParallel 让你的模型并行运行，你可以很容易的在多 GPU 上运行你的操作: model = nn.DataParallel(model)

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [10]:
input_size = 5 
output_size = 2
batch_size = 30 
data_size = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
# 随机生成一些实验数据
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
    def __getitem__(self, index): 
        return self.data[index]
    def __len__(self): 
        return self.len
    
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),batch_size=batch_size, shuffle=True)

In [13]:
# 定义一个简单的线性网络
class Model(nn.Module): # Our model
    def __init__(self, input_size, output_size): 
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(), "output size", output.size())
        return output

In [14]:
# 创建模型，检查可用的GPU数，用nn.DataParallel包裹模型，然后将模型放到GPU上
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [15]:
# 运行模型，如果你没有 GPU 或者只有一个 GPU，当我们获取 30 个输入和 30 个输出，模型将期望获得 30 个 输入和 30 个输出。
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(), "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


In [ ]:
# 但是如果你有多个 GPU，DataParallel自动拆分了你的数据并且将任务单发送到多个 GPU 上。当每一个模型都完成自己的任务之后，DataParallel 收集并且合并这些结果，然后再返回给你
Let‘s use 2 GPUs!
In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2]) Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2]) Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2]) Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2]) Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])